# 7. 프로젝트 - Movielens 영화 추천 실습  
이전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할만한 영화 추천 시스템을 제작해 보겠습니다.  
이번에 활용할 데이터셋은 추천시스템의 MNIST라고 부를만한 Movielens 데이터입니다.  

 * 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
 * 별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해볼 수 있습니다.
 * 별점을 시청횟수로 해석해서 생각하겠습니다.
 * 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.
MovieLens 1M Dataset 데이터셋의 경우는 크기가 작아서 아래와 같이 직접 다운 받고 압축해제 하겠습니다.

MovieLens 1M Dataset 데이터셋의 경우는 크기가 작아서 아래와 같이 직접 다운 받고 압축해제 하겠습니다.

In [ ]:
#1) wget으로 데이터 다운로드
wget http://files.grouplens.org/datasets/movielens/ml-1m.zip

#2) 다운받은 데이터를 작업디렉토리로 이동
mv ml-1m.zip ~/aiffel/recommendata_iu/data

#3) gzip으로 압축된 압축을 해제하면 tar 파일이 하나 나옵니다. 
unzip ml-1m.zip

### 1) 데이터 준비와 전처리
Movielens 데이터는 **rating.dat** 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있습니다.

In [1]:
import pandas as pd
import os


rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head(100)

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
95,2,2490,3,978299966
96,2,1834,4,978298813
97,2,3471,5,978298814
98,2,589,4,978299773


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [4]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings.head()

,user_id,movie_id,count,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [6]:
using_cols = ['user_id', 'movie_id', 'count']
ratings = ratings[using_cols]
ratings.head()

,user_id,movie_id,count
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


여기까지가 전처리입니다. 이후에는 이전 스텝에 소개했던 것과 동일한 방식으로 MF model을 구성하여 내가 좋아할 만한 영화를 추천해 볼 수 있습니다.

### 2) 분석해 봅시다.
___
 * ratings에 있는 유니크한 영화 개수
 * rating에 있는 유니크한 사용자 수
 * 가장 인기있는 영화 30개(인기순)

In [7]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [8]:
# rating에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [9]:
n_movie = pd.merge(ratings, movies)
n_movie

,user_id,movie_id,count,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...
836473,5851,3607,5,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026,4,Slaughterhouse (1987),Horror
836475,5854,690,3,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909,4,"Five Wives, Three Secretaries and Me (1998)",Documentary


In [10]:
# 가장 인기있는 영화 30개(인기순)
movie_count = n_movie.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

### 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.
___




In [11]:
# 선호 영화 5가지 추가 전 확인(user_id 기준)
n_movie.sort_values(by='user_id' ,ascending=True).tail()

,user_id,movie_id,count,title,genre
657728,6040,334,4,Vanya on 42nd Street (1994),Drama
393446,6040,1294,4,M*A*S*H (1970),Comedy|War
253075,6040,994,3,Big Night (1996),Drama
127665,6040,2396,3,Shakespeare in Love (1998),Comedy|Romance
738957,6040,2725,4,Twin Falls Idaho (1999),Drama


In [12]:
my_favorite = ['American Beauty (1999)' , 'Vacation (1983)' ,'Tequila Sunrise (1988)' ,'Source, The (1999)' ,'Male and Female (1919)']

my_playlist = pd.DataFrame({'user_id': ['abc']*5, 'title': my_favorite, 'count':[30]*5})

if not n_movie.isin({'user_id':['abc']})['user_id'].any():
    n_movie = n_movie.append(my_playlist)
    
n_movie.tail(10)

,user_id,movie_id,count,title,genre
836473,5851,3607.0,5,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026.0,4,Slaughterhouse (1987),Horror
836475,5854,690.0,3,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909.0,4,"Five Wives, Three Secretaries and Me (1998)",Documentary
836477,5948,1360.0,5,Identification of a Woman (Identificazione di ...,Drama
0,abc,NaN,30,American Beauty (1999),NaN
1,abc,NaN,30,Vacation (1983),NaN
2,abc,NaN,30,Tequila Sunrise (1988),NaN
3,abc,NaN,30,"Source, The (1999)",NaN
4,abc,NaN,30,Male and Female (1919),NaN


In [13]:
n_idx_movie = pd.merge(n_movie, movies, on=['title'])

n_idx_movie.tail(10)

,user_id,movie_id_x,count,title,genre_x,movie_id_y,genre_y
836473,5494,3530.0,4,Smoking/No Smoking (1993),Comedy,3530,Comedy
836474,5556,2198.0,3,Modulations (1998),Documentary,2198,Documentary
836475,5949,2198.0,5,Modulations (1998),Documentary,2198,Documentary
836476,5675,2703.0,3,Broken Vessels (1998),Drama,2703,Drama
836477,5717,2258.0,4,Master Ninja I (1984),Action,2258,Action
836478,5851,3607.0,5,One Little Indian (1973),Comedy|Drama|Western,3607,Comedy|Drama|Western
836479,5854,3026.0,4,Slaughterhouse (1987),Horror,3026,Horror
836480,5854,690.0,3,"Promise, The (Versprechen, Das) (1994)",Romance,690,Romance
836481,5938,2909.0,4,"Five Wives, Three Secretaries and Me (1998)",Documentary,2909,Documentary
836482,5948,1360.0,5,Identification of a Woman (Identificazione di ...,Drama,1360,Drama


In [14]:
del  n_idx_movie['movie_id_y'], n_idx_movie['genre_y']

n_idx_movie.rename(columns={"movie_id_x":"movie_id", 'genre_x' : 'genre'}, inplace = True)

n_idx_movie.tail(10)

,user_id,movie_id,count,title,genre
836473,5494,3530.0,4,Smoking/No Smoking (1993),Comedy
836474,5556,2198.0,3,Modulations (1998),Documentary
836475,5949,2198.0,5,Modulations (1998),Documentary
836476,5675,2703.0,3,Broken Vessels (1998),Drama
836477,5717,2258.0,4,Master Ninja I (1984),Action
836478,5851,3607.0,5,One Little Indian (1973),Comedy|Drama|Western
836479,5854,3026.0,4,Slaughterhouse (1987),Horror
836480,5854,690.0,3,"Promise, The (Versprechen, Das) (1994)",Romance
836481,5938,2909.0,4,"Five Wives, Three Secretaries and Me (1998)",Documentary
836482,5948,1360.0,5,Identification of a Woman (Identificazione di ...,Drama


In [15]:
user_unique = n_idx_movie['user_id'].unique()
movie_title = n_idx_movie['title'].unique()

user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_title)}

In [16]:
print(user_to_idx['abc']) 
print(movie_to_idx['Vacation (1983)'])

5985
638


In [17]:
temp_user_movie = n_idx_movie['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_movie) == len(n_idx_movie):
    print('user_id column indexing OK!!')
    n_idx_movie['user_id'] = temp_user_movie 
else:
    print('user_id column indexing Fail!!')

temp_movie_title = n_idx_movie['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_title) == len(n_idx_movie):
    print('title column indexing OK!!')
    n_idx_movie['movie_id'] = temp_movie_title
else:
    print('movie_id column indexing Fail!!')

n_idx_movie.tail(10)

user_id column indexing OK!!
title column indexing OK!!


,user_id,movie_id,count,title,genre
836473,1508,3619,4,Smoking/No Smoking (1993),Comedy
836474,1532,3620,3,Modulations (1998),Documentary
836475,1649,3620,5,Modulations (1998),Documentary
836476,1564,3621,3,Broken Vessels (1998),Drama
836477,1574,3622,4,Master Ninja I (1984),Action
836478,1621,3623,5,One Little Indian (1973),Comedy|Drama|Western
836479,3481,3624,4,Slaughterhouse (1987),Horror
836480,3481,3625,3,"Promise, The (Versprechen, Das) (1994)",Romance
836481,4159,3626,4,"Five Wives, Three Secretaries and Me (1998)",Documentary
836482,1648,3627,5,Identification of a Woman (Identificazione di ...,Drama


### 4) CSR matrix를 직접 만들어 봅시다.
___

In [18]:
from scipy.sparse import csr_matrix

num_user = n_idx_movie['user_id'].nunique()
num_movie = n_idx_movie['movie_id'].nunique()



In [19]:
csr_data = csr_matrix((n_idx_movie['count'], (n_idx_movie.user_id, n_idx_movie.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

### 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.
___




In [20]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [21]:
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [22]:
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [23]:
als_model.fit(csr_data_transpose)

### 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.
___




In [24]:
favorite_movie = 'Modulations (1998)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(3620, 0.00896373),
 (3380, 0.0046084207),
 (810, 0.004090737),
 (3149, 0.0040664626),
 (2398, 0.0039431625),
 (1935, 0.0038883453),
 (2527, 0.0038829641),
 (3273, 0.0038440765),
 (724, 0.0038353014),
 (3222, 0.0038350218),
 (3519, 0.0037659472),
 (2405, 0.0037508751),
 (1957, 0.0037331795),
 (3090, 0.0037027162),
 (2974, 0.0036885594)]

In [25]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Modulations (1998)',
 'Deceiver (1997)',
 '54 (1998)',
 'Land Before Time III: The Time of the Great Giving (1995)',
 'Tigerland (2000)',
 'SLC Punk! (1998)',
 'Get Bruce (1999)',
 'Whipped (2000)',
 'Half Baked (1998)',
 'Jerry Springer: Ringmaster (1998)',
 '1-900 (1994)',
 'Bliss (1997)',
 'Body Shots (1999)',
 'Fausto (1993)',
 'Talking About Sex (1994)']

In [26]:
abc, Modulations = user_to_idx['abc'], movie_to_idx['Modulations (1998)']
abc_vector, Modulations_vector = als_model.user_factors[abc], als_model.item_factors[Modulations]

np.dot(abc_vector, Modulations_vector)

0.021708723

### 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.
___



In [27]:
favorite_movie = 'American Beauty (1999)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(99, 0.24122731),
 (170, 0.17183824),
 (126, 0.1064642),
 (271, 0.09723981),
 (51, 0.09057664),
 (793, 0.08585041),
 (121, 0.0821887),
 (222, 0.08191326),
 (3517, 0.08052817),
 (3466, 0.08006931),
 (38, 0.07702927),
 (3573, 0.07633019),
 (107, 0.076164596),
 (175, 0.07537109),
 (48, 0.07517487)]

In [28]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['American Beauty (1999)',
 'Being John Malkovich (1999)',
 'Shakespeare in Love (1998)',
 'Election (1999)',
 'Fargo (1996)',
 'American Pie (1999)',
 'Silence of the Lambs, The (1991)',
 'Pulp Fiction (1994)',
 'Paralyzing Fear: The Story of Polio in America, A (1998)',
 'Schlafes Bruder (Brother of Sleep) (1995)',
 'Sixth Sense, The (1999)',
 'Male and Female (1919)',
 'Jurassic Park (1993)',
 'Men in Black (1997)',
 'Saving Private Ryan (1998)']

### 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.
___

In [29]:
user = user_to_idx['abc']

movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(1194, 0.6628664),
 (1159, 0.49925676),
 (182, 0.46413746),
 (1147, 0.44331947),
 (0, 0.44275576),
 (170, 0.4351142),
 (107, 0.4098456),
 (793, 0.4081021),
 (1155, 0.39979893),
 (1067, 0.38733053),
 (50, 0.38370723),
 (7, 0.37652928),
 (553, 0.35590708),
 (332, 0.3508932),
 (221, 0.34892172),
 (667, 0.3285367),
 (60, 0.32527366),
 (475, 0.32325506),
 (450, 0.32086277),
 (745, 0.31977427)]

In [30]:
[idx_to_movie[i[0]] for i in movie_recommended]

['Fatal Attraction (1987)',
 'Stripes (1981)',
 'Caddyshack (1980)',
 'Sixteen Candles (1984)',
 "One Flew Over the Cuckoo's Nest (1975)",
 'Being John Malkovich (1999)',
 'Jurassic Park (1993)',
 'American Pie (1999)',
 'Risky Business (1983)',
 'Fast Times at Ridgemont High (1982)',
 'Toy Story 2 (1999)',
 'Christmas Story, A (1983)',
 "Pee-wee's Big Adventure (1985)",
 'Moonstruck (1987)',
 'South Park: Bigger, Longer and Uncut (1999)',
 'Fletch (1985)',
 'Star Wars: Episode I - The Phantom Menace (1999)',
 'My Cousin Vinny (1992)',
 'Life Is Beautiful (La Vita � bella) (1997)',
 'Talented Mr. Ripley, The (1999)']